In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch 
import torchvision
import pickle
import numpy as np 
import shutil
import matplotlib.pyplot as plt
import itertools
from pathlib import Path
import fnmatch

import torch.optim
from scipy.io.idl import AttrDict

from PIL import Image


# IMAGE UNZIP

In [ ]:
import zipfile
import os
from os import walk

extract_x64 = False 

if extract_x64: 
  _, _, filenames = next(walk("."))

  for filename in filenames: 
    print(filename)
    with zipfile.ZipFile(filename, 'r') as zip_ref:
      zip_ref.extractall(".")

In [ ]:
import zipfile

extract_x128 = False 
if extract_x128: 
  %cd /content/
  %mkdir flickr
  %mkdir scrape 

  path = "/content/drive/MyDrive/CSC413/flickr/flickr.zip"
  with zipfile.ZipFile(path, 'r') as zip_ref:
      zip_ref.extractall("/content/flickr")
  
  path = "/content/drive/MyDrive/CSC413/scrape/scrape.zip"
  with zipfile.ZipFile(path, 'r') as zip_ref:
      zip_ref.extractall("/content/scrape")


# ARCHITECTURE

In [ ]:
class generator (torch.nn.Module):

  def __init__( self):
    super(generator, self).__init__()
    self.Conv1 = torch.nn.Conv2d(3, 64, 9, padding=4)

    self.residual_block = torch.nn.Sequential()
    for i in range(16):
      self.residual_block = torch.nn.Sequential(*list(self.residual_block) + [ResidualBlock()])

    self.Conv2 = torch.nn.Conv2d(64, 64, 3, padding=1 )
    self.Conv3 = torch.nn.Conv2d(64, 256, 3, padding=1 )
    self.Conv4 = torch.nn.Conv2d(64, 256, 3, padding=1 )
    self.ConvFinal = torch.nn.Conv2d(64, 3, 9, padding=4)
    self.PixelShuffle = torch.nn.PixelShuffle(upscale_factor=2)
    self.PreLU = torch.nn.PReLU();
    self.BatchNorm = torch.nn.BatchNorm2d(num_features=64)


  def forward(self, x ):

    x = self.Conv1(x)
    x = self.PreLU(x)
    
    x1 = x
    x2 = x

    self.residual_block(x1)

    x = self.Conv2(x)
    x = self.BatchNorm(x)
    x = x + x2 

    x = self.Conv3(x)
    x = self.PixelShuffle(x)
    x = self.PreLU(x)
    
    x = self.Conv4(x)
    x = self.PixelShuffle(x)
    x = self.PreLU(x)

    x = self.ConvFinal(x)
    return x 



class ResidualBlock (torch.nn.Module):
  def __init__( self):
    super(ResidualBlock, self).__init__()
    self.block = torch.nn.Sequential(
        torch.nn.Conv2d(64, 64, 3, padding=1),
        torch.nn.BatchNorm2d(64),
        torch.nn.PReLU(),
        torch.nn.Conv2d(64, 64, 3, padding=1),
        torch.nn.BatchNorm2d(64),
    )

  def forward(self, x ):
    return self.block(x) + x 



In [ ]:
class discriminator (torch.nn.Module):

  def __init__(self):
    super(discriminator, self).__init__()
    self.Conv1 = torch.nn.Conv2d(3, 64, 3)
    self.Conv2 = torch.nn.Conv2d(64, 64, 3, stride=2)
    self.BatchNorm2 = torch.nn.BatchNorm2d(num_features=64)
    self.Conv3 = torch.nn.Conv2d(64, 128, 3)
    self.BatchNorm3 = torch.nn.BatchNorm2d(num_features=128)
    self.Conv4 = torch.nn.Conv2d(128, 128, 3, 2)
    self.BatchNorm4 = torch.nn.BatchNorm2d(num_features=128)
    self.Conv5 = torch.nn.Conv2d(128, 256, kernel_size=3)
    self.BatchNorm5 = torch.nn.BatchNorm2d(num_features=256)
    self.Conv6 = torch.nn.Conv2d(256, 256, kernel_size=3,stride=2)
    self.BatchNorm6 = torch.nn.BatchNorm2d(num_features=256)
    self.Conv7 = torch.nn.Conv2d(256, 512, kernel_size=3)
    self.BatchNorm7 = torch.nn.BatchNorm2d(num_features=512)
    self.Conv8 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=2)
    self.BatchNorm8 = torch.nn.BatchNorm2d(num_features=512)

    self.hidden = torch.nn.Conv2d(512, 1024,kernel_size=1,  stride=3)
    self.final_classifier = torch.nn.Conv2d(1024, 1, kernel_size=1)
    self.leakyReLU = torch.nn.LeakyReLU(negative_slope=0.2)
    self.dropout = torch.nn.Dropout(p=0.2)


  def forward(self, x ):
    x = self.leakyReLU(self.Conv1(x))
    x = self.leakyReLU(self.BatchNorm2(self.Conv2(x)))
    x = self.leakyReLU(self.BatchNorm3(self.Conv3(x)))
    x = self.dropout(x)
    x = self.leakyReLU(self.BatchNorm4(self.Conv4(x)))
    x = self.leakyReLU(self.BatchNorm5(self.Conv5(x)))
    x = self.dropout(x)
    x = self.leakyReLU(self.BatchNorm6(self.Conv6(x)))
    x = self.leakyReLU(self.BatchNorm7(self.Conv7(x)))
    x = self.leakyReLU(self.BatchNorm8(self.Conv8(x)))
    x = self.dropout(x)
    x = self.leakyReLU(self.hidden(x))
    x = self.final_classifier(x)
    return torch.sigmoid(x).view(x.shape[0], -1 )

In [ ]:
class vgg_loss ():

  def __init__(self, device):
    super(vgg_loss, self).__init__()
    vgg = torchvision.models.vgg19(pretrained=True)
    vgg = vgg.features

    # Freeze all vgg layers 
    for param in vgg.parameters():
      param.requires_grad = False 
    
    self.vgg = vgg[:27].to(device) 
    self.MSE = torch.nn.MSELoss()

  def __call__(self, input ):
    return self.vgg(input)

# DATA LOADERS



In [ ]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict 

In [ ]:
def get_images_x64(filename):
  d = unpickle(filename)
  x = d['data']
  x = np.dstack((x[:, :4096], x[:, 4096:8192], x[:, 8192:]))
  x = x.reshape((x.shape[0], 64, 64, 3))
  return x

In [ ]:
data_dir = '/content/drive/MyDrive/CSC413/x64/'

class SRDataset(torch.utils.data.Dataset):

  def __init__(self, main_dir):
    self.transform_HR = torchvision.transforms.Compose(
                            [torchvision.transforms.ToTensor(), 
                              torchvision.transforms.RandomCrop(96, pad_if_needed=True, padding_mode="reflect")])

    self.transform_LR = torchvision.transforms.Compose([torchvision.transforms.Resize((24,24), 
                        interpolation=torchvision.transforms.InterpolationMode.BICUBIC)])

    matches = []
    for root, dirnames, filenames in os.walk(main_dir):
      for filename in fnmatch.filter(filenames, '*.jpg'):
        matches.append(os.path.join(root, filename))

    print("Number of images found in " + main_dir + ":", end="")
    print(len(matches))
    self.matches = np.array(matches) 
  
  def __len__(self):
    return len(self.matches)


  def __getitem__(self, idx):
    path = self.matches[idx]
    image = Image.open(path).convert("RGB")
    HR = self.transform_HR(image)
    LR = self.transform_LR(HR)
    return {'HR': HR, 'LR': LR}

# TRAINING CODE

## Helper Functions

In [ ]:
def gan_checkpoint(path, G, D):
    """Saves the parameters of the generator G and discriminator D.
    """
    G_path = os.path.join(path, 'G.pkl')
    D_path = os.path.join(path, 'D.pkl')
    torch.save(G.state_dict(), G_path)
    torch.save(D.state_dict(), D_path)

def load_checkpoint(opts):
    """Loads the generator and discriminator models from checkpoints.
    """
    G_path = os.path.join(opts.load, 'G.pkl')
    D_path = os.path.join(opts.load, 'D.pkl')

    G = generator()
    D = discriminator()

    G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))
    D.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage))

    G.to(opts.device)

    return G, D

In [ ]:
"""
CREDITS: CSC413 PA4 DCGAN
"""

def train(opts):
    """Runs the training loop.
        * Saves checkpoint every opts.checkpoint_every iterations
        * Saves generated samples every opts.sample_every iterations
    """
    # Create generators and discriminators
    G = generator().to(opts.device)
    D = discriminator().to(opts.device)

    g_params = G.parameters()  # Get generator parameters
    d_params = D.parameters()  # Get discriminator parameters

    # Create optimizers for the generators and discriminators
    g_optimizer = torch.optim.Adam(g_params, opts.lr, [opts.beta1, opts.beta2])
    d_optimizer = torch.optim.Adam(d_params, opts.lr, [opts.beta1, opts.beta2])

    print("loading scrape ... ")
    train_set_scrape = SRDataset("/content/flickr/")
    train_loader_scrape = torch.utils.data.DataLoader(train_set_scrape, opts.batch_size,
                              shuffle=True, num_workers=opts.num_workers)

    print("loading flickr ... ")
    train_set_flickr = SRDataset("/content/scrape/")
    train_loader_flickr = torch.utils.data.DataLoader(train_set_flickr, opts.batch_size,
                              shuffle=True, num_workers=opts.num_workers)

    # start with the scraped images 
    train_iter = iter(train_loader_scrape)
    iter_per_epoch = len(train_iter)
    is_scrape = True 

    total_train_iters = opts.train_iters
 
    # adversarial_loss = torch.nn.BCEWithLogitsLoss()
    gp_weight = 1

    losses ={}
    losses['iteration'] = []
    losses['D_loss'] = []
    losses['G_loss'] = []


    VGG = vgg_loss(opts.device)
    MSE = torch.nn.MSELoss()
    BCE = torch.nn.BCEWithLogitsLoss()


    # Quick Testing 
    data_transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize(64),
        torchvision.transforms.ToTensor()
    ])

    image_name = "/content/drive/MyDrive/CSC413/LR_Test.jpg"
    og_image = Image.open(image_name)
    image = data_transforms(og_image).float()
    image = torch.tensor(image, requires_grad=True)
    image = image.unsqueeze(0).to(opts.device)


    try:
        for iteration in range(1, opts.train_iters + 1):

            G.train()
            g_optimizer.zero_grad()

            # Reset data_iter for each epoch
            if iteration % iter_per_epoch == 0:
              if is_scrape: 
                train_iter = iter(train_loader_flickr)
                iter_per_epoch = len(train_iter)
              else: 
                train_iter = iter(train_loader_scrape)
                iter_per_epoch = len(train_iter)
              is_scrape = !is_scrape 
              

            next = train_iter.next()
            HR =  next['HR']
            LR = next['LR']
            HR = HR.to(opts.device)
            LR = LR.to(opts.device)

            # ------------------------ DISCRIMINATOR LOSS ---------------------


            d_optimizer.zero_grad()

            HRE = G(LR)
            logits_HRE = D(HRE)
            logits_HR = D(HR)
            
            D_1 = torch.mean(logits_HR)
            D_2 = torch.mean(logits_HRE)                
            D_loss = 1 - D_1 + D_2 

            if D_loss > 0.001: 
              D_loss.backward(retain_graph=True)
              d_optimizer.step()


            # ------------------------ GENERATOR LOSS -----------------------

            g_optimizer.zero_grad()

            HRE = G(LR)
            logits_HRE = D(HRE)
            vgg_HR = VGG(HR)
            vgg_HRE = VGG(HRE)

            G_1 = torch.mean(1 - logits_HRE)
            G_VGG = MSE(vgg_HRE, vgg_HR)
            G_MSE = MSE(HRE, HR)
            G_loss = G_MSE + 0.006 * G_VGG + 0.001 * G_1 

            G_loss.backward()
            g_optimizer.step()

            # ------------------------  CHECKPOINTS & SAMPLING -------------------

            if iteration % opts.log_step == 0:
                losses['iteration'].append(iteration)
                losses['D_loss'].append(D_loss.item())
                losses['G_loss'].append(G_loss.item())
                print('Iteration [{:4d}/{:4d}] | D_loss: {:6.4f} |  G_loss: {:6.4f}'.format(
                    iteration, total_train_iters, D_loss.item(), G_loss.item()))

            # Save the model parameters
            if iteration % opts.checkpoint_every == 0:
                checkpoint = opts.checkpoint

                dir = checkpoint + str(iteration)
                if os.path.exists(dir):
                  shutil.rmtree(dir)
                os.mkdir(dir)
                gan_checkpoint(dir, G, D)

                G.eval()

                image_est = (G(image).clone().detach().cpu().numpy())
                image_est = np.squeeze(image_est, axis=0)
                image_est = np.transpose(image_est, (1,2,0) )

                f, axarr = plt.subplots(1,2)
                axarr[0].imshow(og_image)
                axarr[1].imshow(image_est)
                f.savefig(dir + '/sample.png', bbox_inches='tight')

    except KeyboardInterrupt:
        print('Exiting early from training.')
        return G, D

    return G, D

## Hyperparameters and running loop

In [ ]:
opts = AttrDict()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

args_dict = {
    'checkpoint': "/content/drive/MyDrive/CSC413/checkpoints/",
    'lr': 0.00005,
    'beta1':0.9,
    'beta2':0.999,
    'batch_size': 32,
    'device': device, 
    'epochs': 12,
    'num_workers': 4,
    'resume': False,
    'log_step': 100, 
    'checkpoint_every':1000,
    'train_iters':200000,
    'batch' : 'train_data_batch_1'
}

torch.autograd.set_detect_anomaly(True)
opts.update(args_dict)
train(opts)

# TEST IMAGE CODE

In [ ]:
import cv2

def torch_to_saveable_image(image_est):
  image_est = image_est.cpu().clone().detach().numpy()
  image_est = np.transpose(image_est, (1,2,0))
  image_est = np.clip(image_est, a_min=0, a_max=1)
  print(np.min(image_est))
  print(np.max(image_est))

  image_est = cv2.normalize(image_est, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
  image_est = cv2.cvtColor(image_est, cv2.COLOR_RGB2BGR)
  return image_est


use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

first_path = "/content/drive/MyDrive/CSC413/final/x64.pkl"
Gx64 = generator()
Gx64.load_state_dict(torch.load(first_path, map_location=lambda storage, loc: storage))

second_path = "/content/drive/MyDrive/CSC413/final/x128.pkl"
Gx128 = generator()
Gx128.load_state_dict(torch.load(second_path, map_location=lambda storage, loc: storage))

Gx128.to(device)
Gx64.to(device)


data_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((64,64))
])


filetype = ".png"
directory = "/content/drive/MyDrive/CSC413/set14/"

from os import walk

_, _, filenames = next(walk(directory))

filenames = ["monarch"]
for name in filenames:
  
  filename = os.path.splitext(name)[0]
  image_name = directory + filename  + filetype


  og_image = Image.open(image_name).convert("RGB")
  width, height = og_image.size

  data_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((height // 4 ,width // 4 ))
  ])
  

  image = data_transforms(og_image).float()
  image = torch.tensor(image, requires_grad=False)
  image = image.unsqueeze(0).to(device)


  image_est = Gx128(image).squeeze(0)


  bicubic = torchvision.transforms.Resize((height,width))
  image =  image.squeeze(0)
  bicubic = bicubic(image)
  bicubic = torch_to_saveable_image(bicubic)


  image_est = torch_to_saveable_image(image_est)
  cv2.imwrite(directory + filename + "_GAN.png", image_est)
  cv2.imwrite(directory + filename + "_Bicubic.png", bicubic)

  # image_est = Gx128(image).squeeze(0)
  # image_est = torch_to_saveable_image(image_est)
  # cv2.imwrite(directory + filename + "_x128.png", image_est)
